# 축구 A매치 조건 별 승패 예측



##1. 데이터 가져오기 및 데이터 정리



In [88]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [89]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/results.csv')

date 컬럼을 'datetime'타입으로 변경합니다.<br>

선수 생명을 고려해서 2000년 이후의 데이터만 사용합니다.


In [90]:
df['date'] = pd.to_datetime(df['date'])
df = df[df['date'] > '2000-01-01']

In [91]:
df.head(30)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
22619,2000-01-04,Egypt,Togo,2,1,Friendly,Aswan,Egypt,False
22620,2000-01-07,Tunisia,Togo,7,0,Friendly,Tunis,Tunisia,False
22621,2000-01-08,Trinidad and Tobago,Canada,0,0,Friendly,Port of Spain,Trinidad and Tobago,False
22622,2000-01-09,Burkina Faso,Gabon,1,1,Friendly,Ouagadougou,Burkina Faso,False
22623,2000-01-09,Guatemala,Armenia,1,1,Friendly,Los Angeles,United States,True
22624,2000-01-09,Ivory Coast,Egypt,2,0,Friendly,Abidjan,Ivory Coast,False
22625,2000-01-09,Mexico,Iran,2,1,Friendly,Oakland,United States,True
22626,2000-01-11,Bermuda,Canada,0,2,Friendly,Hamilton,Bermuda,False
22627,2000-01-11,Burkina Faso,Cameroon,2,2,Friendly,Ouagadougou,Burkina Faso,False
22628,2000-01-13,Senegal,Cameroon,0,0,Friendly,Dakar,Senegal,False


## 2. 전처리


### 2.1 승무패 칼럼 생성 
홈팀과 어웨이팀의 점수를 비교하여 홈팀기준 <br>
승리 : 2 <br>
무승부 : 1 <br>
패배 : 0 <br>

을 기록하는 컬럼을 생성합니다.

In [92]:
def hometeam(df):
  if df['home_score'] == df['away_score']:
         val = 1
  elif df['home_score'] > df['away_score']:
         val = 2
  else:
         val = 0
  return val

def awayteam(df):
  if df['home_score'] == df['away_score']:
         val = 1
  elif df['home_score'] < df['away_score']:
         val = 2
  else:
         val = 0
  return val

### 2.2 월드컵에 출전하는 국가의 A매치 경기 기록 만들기
2022년 카타르 월드컵에 출전하는 국가를 countries 리스트에 담았습니다.<br>

카타르 월드컵에 출전하는 출전 국가의 A매치 경기 결과를 파악합니다.<br>

**친선 경기를 제외하고 많은경우 제 3 국가에서 경기를 치르기 때문에 홈 어드벤티지는 고려하지 않습니다.**<br>
<br>
따라서 경기결과를 알고싶은 국가를 모두 홈팀으로 옮겨서 승무패 칼럼을 만듭니다. 아래 nation2 함수를 이용했습니다.

In [93]:
countries = ['Ecuador','Qatar','Senegal','Netherlands','England','Iran','United States','Wales','Argentina',
             'Saudi Arabia','Mexico','Poland','France','Australia','Denmark','Tunisia','Spain','Costa Rica',
             'Germany','Japan', 'Belgium','Canada','Morocco','Croatia','Brazil','Serbia','Switzerland','Cameroon',
             'Portugal','Ghana', 'Uruguay', 'South Korea']

In [94]:
for i, country in enumerate(countries):
    want_to_find = country

    #원하는 팀을 모두 홈으로 옮기고 승점도 부여해주는 함수
    def nation2():
        df1 = df[df['home_team']== want_to_find]
        df1['Target'] = want_to_find 
        df1['result'] = df.apply(hometeam,axis = 1)
    
        df2 = df[df['away_team'] == want_to_find]
        df2 = df2.rename(columns = {'home_team' : 'away_team', 'away_team' : 'home_team'}) # X 팀의 경기결과를 어웨이에 있는 X 팀을 홈팀으로 옮깁니다.
        df2['Target'] = want_to_find
        df2['result'] = df.apply(hometeam,axis = 1)
        df_f = pd.concat([df1,df2]).reset_index(drop=True) # X 팀의 모든 경기 결과를 얻을 수 있습니다. 
        return df_f

    #add_[숫자]로 저장
    globals()['add_{}'.format(i)] = nation2()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [95]:
# 32개 국가의 매치 기록을 total_df 객체로 생성
list=[]
for i in range(32):
  list.append(globals()['add_'+str(i)])

total_df = pd.concat(list)

### 2.3 피파랭킹 데이터 가져오기


In [96]:
ranking = pd.read_excel('/content/drive/MyDrive/fifa_ranking.xlsx', engine = 'openpyxl')

In [97]:
ranking.head(30)

,FIFA MEN'S RANKING,COUNTRY,POINTS,PREVIOUS
0,1,Brazil,1832.69,1823.42
1,2,Belgium,1827.00,1828.45
2,3,France,1789.85,1786.15
3,4,Argentina,1765.13,1766.99
4,5,England,1761.71,1755.52
5,6,Italy,1723.31,1740.77
6,7,Spain,1709.19,1704.75
7,8,Portugal,1674.78,1660.25
8,9,Mexico,1658.82,1647.90
9,10,Netherlands,1658.66,1653.73


In [98]:
total_df_prac = total_df.copy()
away_list = total_df_prac['away_team'].tolist()

In [99]:
final_df = total_df_prac

In [100]:
#fifa_country_data
team_value = pd.read_excel('/content/drive/MyDrive/fifa_country_data.xlsx', engine = 'openpyxl')

In [101]:
##몸값 데이터
team_value

,#,Nation,Squad_size,Avg_age,Total_value,Confederation,Points
0,1,Brazil,28,27.6,€1.01bn,CONMEBOL,1838
1,2,Belgium,24,28.5,€339.00m,UEFA,1822
2,3,Argentina,28,28.0,€627.00m,CONMEBOL,1771
3,4,France,25,26.9,€1.06bn,UEFA,1765
4,5,England,27,25.7,€1.29bn,UEFA,1737
...,...,...,...,...,...,...,...
206,207,Sri Lanka,23,26.1,€1.01m,AFC,825
207,208,US Virgin,29,23.8,€100Th.,CONCACAF,824
208,209,B. Virgin,24,22.0,-,CONCACAF,809
209,210,Anguilla,22,26.4,-,CONCACAF,791


In [102]:
nations = team_value['Nation'].tolist()
nation = []
for i in range(len(nations)):
  nation.append(nations[i][1:])
########## nation 리스트에 값이 [%Brazil, %Belgium ...] 이런식이라 뒤에 [1:]으로 국가명만 뽑음


squad_size = team_value['Squad_size'].tolist()
Avg_age = team_value['Avg_age'].tolist()
Total_value = team_value['Total_value'].tolist()


In [103]:
final_df['away_team'].unique()

array(['Honduras', 'Venezuela', 'Panama', 'Peru', 'Colombia', 'Bolivia',
       'Chile', 'Brazil', 'Paraguay', 'Argentina', 'Uruguay', 'Guatemala',
       'Haiti', 'South Africa', 'Serbia', 'Senegal', 'Croatia',
       'Costa Rica', 'Estonia', 'Jordan', 'Italy', 'Poland', 'Uganda',
       'Sweden', 'Republic of Ireland', 'El Salvador', 'Jamaica',
       'Greece', 'Germany', 'Spain', 'England', 'France',
       'Trinidad and Tobago', 'Japan', 'Nigeria', 'Cape Verde', 'Mexico',
       'United States', 'Canada', 'Turkey', 'Bulgaria', 'Catalonia',
       'Netherlands', 'North Macedonia', 'Galicia', 'Iran', 'Oman',
       'South Korea', 'Portugal', 'Australia', 'Switzerland', 'Qatar',
       'Bosnia and Herzegovina', 'Wales', 'Sudan', 'Palestine',
       'Kazakhstan', 'Pakistan', 'Kuwait', 'Egypt', 'Thailand',
       'Uzbekistan', 'Malaysia', 'Hong Kong', 'Bahrain', 'China PR',
       'Iraq', 'United Arab Emirates', 'Zimbabwe', 'Singapore', 'Yemen',
       'Turkmenistan', 'Kyrgyzstan', 'Lao

In [104]:
nation = ['Brazil',
 'Belgium',
 'Argentina',
 'France',
 'England',
 'Spain',
 'Italy',
 'Netherlands',
 'Portugal',
 'Denmark',
 'Germany',
 'Mexico',
 'Uruguay',
 'United States',
 'Croatia',
 'Switzerland',
 'Colombia',
 'Senegal',
 'Wales',
 'Sweden',
 'Peru',
 'Morocco',
 'Iran',
 'Japan',
 'Serbia',
 'Poland',
 'Ukraine',
 'South Korea',
 'Chile',
 'Tunisia',
 'Nigeria',
 'Czechia',
 'Austria',
 'Costa Rica',
 'Russia',
 'Norway',
 'Hungary',
 'Cameroon',
 'Australia',
 'Egypt',
 'Algeria',
 'Türkiye',
 'Canada',
 'Ecuador',
 'Scotland',
 'Mali',
 'Republic of Ireland',
 'Greece',
 'Qatar',
 'Paraguay',
 'Slovakia',
 'Ivory Coast',
 'Saudi Arabia',
 'Romania',
 'Burkina Faso',
 'Venezuela',
 'Bosnia',
 'N. Ireland',
 'Finland',
 'Ghana',
 'Panama',
 'Jamaica',
 'Iceland',
 'North Macedonia',
 'Slovenia',
 'Albania',
 'Montenegro',
 'South Africa',
 'U. A. E.',
 'Iraq',
 'El Salvador',
 'Cape Verde',
 'DR Congo',
 'Bulgaria',
 'Oman',
 'Israel',
 'Uzbekistan',
 'China',
 'Gabon',
 'Honduras',
 'Bolivia',
 'Georgia',
 'Guinea',
 'Curaçao',
 'Bahrain',
 'Jordan',
 'Haiti',
 'Zambia',
 'Syria',
 'Uganda',
 'Benin',
 'Armenia',
 'Luxembourg',
 'Palestine',
 'Kyrgyzstan',
 'Belarus',
 'Vietnam',
 'Equat. Guinea',
 'Congo',
 'Lebanon',
 'Trinidad',
 'Kenya',
 'New Zealand',
 'India',
 'Madagascar',
 'Kosovo',
 'Cyprus',
 'Tajikistan',
 'Estonia',
 'Mauritania',
 'Thailand',
 'North Korea',
 'Sierra Leone',
 'Kazakhstan',
 'Guinea-Bissau',
 'Guatemala',
 'Namibia',
 'Mozambique',
 'Niger',
 'Libya',
 'Malawi',
 'Angola',
 'Zimbabwe',
 'The Gambia',
 'Faroe Islands',
 'Comoros',
 'Togo',
 'Azerbaijan',
 'Latvia',
 'Sudan',
 'Tanzania',
 'Antigua and B.',
 'Central Africa',
 'Philippines',
 'Turkmenistan',
 'Rwanda',
 'Solomon Islands',
 'Ethiopia',
 'Nicaragua',
 'Burundi',
 'St. Kitts/Nevis',
 'Lithuania',
 'Suriname',
 'Eswatini',
 'Hong Kong',
 'Lesotho',
 'Malaysia',
 'Kuwait',
 'Botswana',
 'Liberia',
 'Dominican Rep.',
 'Andorra',
 'Yemen',
 'Afghanistan',
 'Indonesia',
 'Maldives',
 'C. Taipei',
 'Myanmar',
 'Singapore',
 'New Caledonia',
 'Papua N. Guinea',
 'Tahiti',
 'Fiji',
 'Vanuatu',
 'South Sudan',
 'Barbados',
 'Cuba',
 'Malta',
 'Puerto Rico',
 'Bermuda',
 'Grenada',
 'Guyana',
 'Saint Lucia',
 'Cambodia',
 'Belize',
 'Nepal',
 'Montserrat',
 'Moldova',
 'Mauritius',
 'St. Vincent',
 'Chad',
 'Macau',
 'Laos',
 'Mongolia',
 'Dominica',
 'Bhutan',
 'São Tomé and P.',
 'A. Samoa',
 'Cook Islands',
 'Brunei',
 'Samoa',
 'Bangladesh',
 'Djibouti',
 'Liechtenstein',
 'Seychelles',
 'Pakistan',
 'Cayman Islands',
 'Tonga',
 'Timor-Leste',
 'Somalia',
 'Gibraltar',
 'Eritrea',
 'Aruba',
 'Bahamas',
 'Guam',
 'Turks-Caicos',
 'Sri Lanka',
 'US Virgin',
 'B. Virgin',
 'Anguilla',
 'San Marino']


In [105]:
home_index = []
# 홈팀 피파랭킹 순위 리스트 만들기
for i in final_df['home_team']:
   index_num = nation.index(i)
   home_index.append(index_num)




In [106]:
away_index = []
# 어웨이팀 피파랭킹 순위 리스트 만들기
# 어웨이팀 중 국가 리스트에 안들어있거나 이름이 다른경우 평균정도되는 피파랭킹 100위 부여
for i in range(len(final_df['away_team'])):
  try:
   index_num2 = nation.index(final_df['away_team'][i])
  except:
   index_num2 = 100
  
  away_index.append(index_num2)



In [107]:
column_home_rank = []
column_home_squad_size = []
column_home_avg_age = []
column_home_total_value = []

for i in home_index:
  column_home_rank_for = team_value['#'][i]
  column_home_squad_size_for = team_value["Squad_size"][i]
  column_home_avg_age_for = team_value['Avg_age'][i]
  column_home_total_value_for = team_value['Total_value'][i]

  column_home_rank.append(column_home_rank_for)
  column_home_squad_size.append(column_home_squad_size_for)
  column_home_avg_age.append(column_home_avg_age_for)
  column_home_total_value.append(column_home_total_value_for)

final_df.insert(loc=2,column='home_rank', value = column_home_rank)
final_df.insert(loc=3,column='home_squad_size', value = column_home_squad_size)
final_df.insert(loc=4,column='home_avg_age', value = column_home_avg_age)
final_df.insert(loc=5,column='home_total_value', value = column_home_total_value)

In [108]:
column_away_rank =[]
column_away_squad_size = []
column_away_avg_age = []
column_away_total_value = []

for i in away_index: # away 팀의 피파랭킹, 선수단 크기, 선수의 평균 나이, 가치 리스트 생성 
  column_away_rank_for = team_value['#'][i]
  column_away_squad_size_for = team_value["Squad_size"][i]
  column_away_avg_age_for = team_value['Avg_age'][i]
  column_away_total_value_for = team_value['Total_value'][i]


  column_away_rank.append(column_away_rank_for)
  column_away_squad_size.append(column_away_squad_size_for)
  column_away_avg_age.append(column_away_avg_age_for)
  column_away_total_value.append(column_away_total_value_for)

# findal_df 데이터프레임에 컬럼으로 각각 삽입
final_df.insert(loc=7,column='away_rank', value = column_away_rank)
final_df.insert(loc=8,column='away_squad_size', value = column_away_squad_size)
final_df.insert(loc=9,column='away_avg_age', value = column_away_avg_age)
final_df.insert(loc=10,column='away_total_value', value = column_away_total_value)

### 2.4 필요없는 칼럼 삭제

- 시계열 모델링이 아니기 때문에 date 칼럼 삭제
- 'tournament' 삭제
- 'city' 삭제
- 'country' 삭제
- 'neutral' 삭제

In [109]:
final_df.drop(columns = ['date','tournament', 'city', 'country', 'neutral'],inplace = True)

### 2.5 선수단 값 단위 통일

Th 대신 00을 넣어도 상관없는 이유 => Th데이터들은 소수점이 없기 때문
<br>


In [110]:
###몸값 칼럼 두개 (home,away) 단위 통일 => astype('float')
final_df['home_total_value'] = final_df['home_total_value'].str.replace("€","")
final_df['home_total_value'] = final_df['home_total_value'].str.replace("m","000")
final_df['home_total_value'] = final_df['home_total_value'].str.replace("bn","000000")
final_df['home_total_value'] = final_df['home_total_value'].str.split(".")
final_df['home_total_value'] = final_df['home_total_value'].str[0] + final_df['home_total_value'].str[1]
final_df['home_total_value'] = final_df['home_total_value'].str[:-2] + "." + final_df['home_total_value'].str[-2:]
final_df['home_total_value'] = final_df['home_total_value'].astype('float')

final_df['away_total_value'] = final_df['away_total_value'].str.replace("€","")
final_df['away_total_value'] = final_df['away_total_value'].str.replace("m","000")
final_df['away_total_value'] = final_df['away_total_value'].str.replace("bn","000000")
final_df['away_total_value'] = final_df['away_total_value'].str.replace("Th","00")
final_df['away_total_value'] = final_df['away_total_value'].str.split(".")
final_df['away_total_value'] = final_df['away_total_value'].str[0] + final_df['away_total_value'].str[1]
final_df['away_total_value'] = final_df['away_total_value'].str[:-2] + "." + final_df['away_total_value'].str[-2:]
final_df['away_total_value'] = final_df['away_total_value'].astype('float')


In [111]:
# final_df['away_total_value'][final_df['away_total_value'].str.contains('Th')].unique()

In [112]:
###홈팀, 어웨이팀 이름과 Target 컬럼 없어도 되므로 삭제
final_df.drop(columns=['home_team','away_team','Target'],inplace=True)

In [113]:
# 결측치 제거
final_df=final_df.dropna()

In [114]:
# 데이터 타입 정리 
final_df = final_df.astype({'home_rank':'int','away_rank':'int','away_squad_size':'int'})

In [115]:
final_df.dtypes

home_rank             int64
home_squad_size       int64
home_avg_age        float64
home_total_value    float64
away_rank             int64
away_squad_size       int64
away_avg_age        float64
away_total_value    float64
home_score            int64
away_score            int64
result                int64
dtype: object

###2.6 전처리 완료 데이터 생성

In [116]:
final_df.to_csv('fifa.csv', index=False)

## 모델링

In [117]:
df = pd.read_csv('/content/fifa.csv')

In [118]:
df.head()

,home_rank,home_squad_size,home_avg_age,home_total_value,away_rank,away_squad_size,away_avg_age,away_total_value,home_score,away_score,result
0,44,28,25.7,101750.0,101,25,26.7,6750.0,1,3,0
1,44,28,25.7,101750.0,101,25,26.7,6750.0,2,0,2
2,44,28,25.7,101750.0,101,25,26.7,6750.0,5,0,2
3,44,28,25.7,101750.0,101,25,26.7,6750.0,2,1,2
4,44,28,25.7,101750.0,101,25,26.7,6750.0,0,0,1


In [119]:
import numpy as np 

y = np.array(pd.get_dummies(df['result']))
X = np.array(df.drop(columns = ['result','home_score','away_score'], axis=1))

In [120]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [121]:
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

In [122]:
model = Sequential()
model.add(Dense(16, input_dim=8, activation='relu'))
model.add(Dense(3, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [123]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 16)                144       
                                                                 
 dense_13 (Dense)            (None, 3)                 51        
                                                                 
Total params: 195
Trainable params: 195
Non-trainable params: 0
_________________________________________________________________


In [124]:
model.fit(X_train, y_train , epochs=60, batch_size = 100)

Epoch 1/60
74/74 [==============================] - 1s 2ms/step - loss: 179591.8906 - accuracy: 0.2888
Epoch 2/60
74/74 [==============================] - 0s 2ms/step - loss: 23911.1934 - accuracy: 0.3089
Epoch 3/60
74/74 [==============================] - 0s 2ms/step - loss: 1599.0160 - accuracy: 0.3042
Epoch 4/60
74/74 [==============================] - 0s 2ms/step - loss: 739.4258 - accuracy: 0.3048
Epoch 5/60
74/74 [==============================] - 0s 2ms/step - loss: 306.2905 - accuracy: 0.2970
Epoch 6/60
74/74 [==============================] - 0s 2ms/step - loss: 291.2588 - accuracy: 0.3226
Epoch 7/60
74/74 [==============================] - 0s 2ms/step - loss: 360.5502 - accuracy: 0.3449
Epoch 8/60
74/74 [==============================] - 0s 2ms/step - loss: 281.4882 - accuracy: 0.3728
Epoch 9/60
74/74 [==============================] - 0s 2ms/step - loss: 480.0817 - accuracy: 0.3668
Epoch 10/60
74/74 [==============================] - 0s 2ms/step - loss: 274.2963 - accuracy: 

In [125]:
pred = model.predict(X_test)
print(pred)
pd.DataFrame(pred).round(2)

58/58 [==============================] - 0s 1ms/step
[[0.0000000e+00 9.9999994e-01 0.0000000e+00]
 [0.0000000e+00 9.9999994e-01 0.0000000e+00]
 [0.0000000e+00 9.9999994e-01 0.0000000e+00]
 ...
 [0.0000000e+00 9.9999994e-01 5.0831594e-33]
 [0.0000000e+00 9.9999994e-01 0.0000000e+00]
 [0.0000000e+00 1.0000000e+00 0.0000000e+00]]


,0,1,2
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,0.0,1.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0
...,...,...,...
1836,0.0,1.0,0.0
1837,0.0,1.0,0.0
1838,0.0,1.0,0.0
1839,0.0,1.0,0.0


In [126]:
model.evaluate(X_test, y_test)

58/58 [==============================] - 0s 2ms/step - loss: 575.4808 - accuracy: 0.2363


[575.4807739257812, 0.23628462851047516]

In [127]:
model.save('worldcup_modelv1.h5')

In [128]:
worldcup_model = keras.models.load_model('worldcup_modelv1.h5')

In [129]:
worldcup_model.predict(X_test)

58/58 [==============================] - 0s 1ms/step


array([[0.0000000e+00, 9.9999994e-01, 0.0000000e+00],
       [0.0000000e+00, 9.9999994e-01, 0.0000000e+00],
       [0.0000000e+00, 9.9999994e-01, 0.0000000e+00],
       ...,
       [0.0000000e+00, 9.9999994e-01, 5.0831594e-33],
       [0.0000000e+00, 9.9999994e-01, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00]], dtype=float32)

In [130]:
model.evaluate(X_test, y_test)

58/58 [==============================] - 0s 2ms/step - loss: 575.4808 - accuracy: 0.2363


[575.4807739257812, 0.23628462851047516]

In [131]:
X = np.array([[50, 20, 25, 10000, 20, 20, 30, 20000]])

In [132]:
worldcup_model.predict(X)

1/1 [==============================] - 0s 63ms/step


array([[0., 0., 1.]], dtype=float32)